# 1 - CSV de la base IMDB et imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# namebasics_df = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t', on_bad_lines='skip', low_memory=False)
# titleakas_df = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', on_bad_lines='skip', low_memory=False)
titlebasics_df = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', on_bad_lines='skip',low_memory=False, nrows=50)
# titlecrew_df = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep='\t', on_bad_lines='skip', low_memory=False)
# titleprincipals_df = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep='\t', on_bad_lines='skip', low_memory=False)
# titleratings_df = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', on_bad_lines='skip', low_memory=False)

# 2 - Création de fonctions d'inspection

In [ ]:
# Module d'inspection

def valeurs_isna(df):
    return df.isna().sum()

def inspection(df):
    return df.shape, df.columns, df.describe, df.info

:# 3 - Nettoyage et exploration de Title Basics

---

Nous avons commencé par le nettoyage de cette table, qui contenait beaucoup d'informations concernant les films. 
Nous avons décidé d'enlever : 

*   titleType : tout ce qui n'est pas "Movie"
*   isAdult : tout ce qui est 1 
*   startYear : ce qui est sorti avant 1930 (nous avons considéré cette date intéressante car elle permet d'avoir des classiques d'une durée de long métrage habituelle (+ de 58 min), et du cinéma parlant)
*   endYear : nous avons enlevé cette colonne
*   runtimeMinutes : les films de moins de 58 min sont enlevés car ce ne sont pas des longs métrages



1 - Explorations basiques

In [ ]:
titlebasics_df.columns

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')

In [ ]:
titlebasics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


2 - Gestion de runtimeMinutes

In [ ]:
# Conversion de runtimeMinutes en Float. Attention il reste des NaN !
titlebasics_df['runtimeMinutes'] = pd.to_numeric(titlebasics_df['runtimeMinutes'], errors='coerce')

# Tri de runtimeMinutes de plus de 58 min
titlebasics_df_runtime = titlebasics_df.drop(titlebasics_df[titlebasics_df['runtimeMinutes'] <= 58].index, inplace = False)

3 - Gestion de titleType

In [ ]:
# Tri de titleType = movie
titlebasics_df_titletype = titlebasics_df_runtime.drop(titlebasics_df_runtime[titlebasics_df_runtime['titleType'] != 'movie'].index, inplace = False)

4 - Gestion de isAdult

In [ ]:
# Tri de isAdult 
titlebasics_df_isadult = titlebasics_df_titletype.drop(titlebasics_df_titletype[titlebasics_df_titletype['isAdult'] == 1].index, inplace = False)

5 - Gestion de startYear

In [ ]:
# Tri de date de sortie (conversion en numérique d'abord. Attention il reste des Nan !) après 1930
titlebasics_df_isadult['startYear'] = pd.to_numeric(titlebasics_df_isadult['startYear'], errors='coerce')
titlebasics_df_startyear = titlebasics_df_isadult.drop(titlebasics_df_isadult[titlebasics_df_isadult['startYear'] <= 1930].index, inplace = False)

5 - Suppression de la colonne endYear

In [ ]:
titlebasics_df_startyear.drop(column='endYear', inplace = True)

6 - Sortie du CSV qui servira de base pour les merge

In [ ]:
# Enregistrement du CSV (nouvelle base) 
titlebasics_df_startyear.to_csv('df_titlebasics_v3.csv', index=False)

# 3 - Merge des tables contenant les nconsts 

Nous avons déjà mergé ces deux tables contenant les informations sur les équipes de films, qui seront ensuite mergées de nouveau avec les autres tables concernant les films. Lors du merge final, seules les informations sur les films que nous avons décidé de garder seront dans le dataframe. 

In [ ]:
# Merge de title principals et name basics
merge_nconst = pd.merge(titleprincipals_df,
               namebasics_df,
               how='inner',
               left_on='nconst',
               right_on='nconst')

# Sortie en CSV
merge_nconst.to_csv('merged_principals_namebasics', index=False)

# Nettoyage des colonnes et données inutiles : colonnes ordering, characters, job and primaryProfession ; categories autres que director, actor, actress)
merged_principals_namebasics = pd.read_csv('merged_principals_namebasics.csv', sep=',', on_bad_lines='skip', low_memory=False)
merged_principals_namebasics_v2 = merged_principals_namebasics.drop('ordering', 'characters', 'job', 'primaryProfession', axis=1)
merged_principals_namebasics_v3 = merged_principals_namebasics_v2.drop(merged_principals_namebasics_v2[merged_principals_namebasics_v2['category'] != 'director' | 'actor' | 'actress'].index, inplace = False)

# Sortie en CSV 
merged_principals_namebasics_v3.to_csv('merged_principals_namebasics_v3.csv', index=False)

In [ ]:
# Pour le merge des autres tables 

# Renommer les colonnes contenant les clés primaires pour faciliter les merges à venir
titlesakas_df.rename(columns = {'titleId':'tconst'}, inplace = True)

# 4 - Choix à partir du CSV mergé de toutes les tables

Nous avons fait plusieurs choix pour les csv finaux :

*   Calculer la médiane du nombre de votes. Celle-ci étant à 673, et la moyenne à plus de 17 000, nous avons décidé de garder tous les films au dessus de la médiane, la moyenne étant trop restrictive. 
*   Séparer le dataframe final en films sortis et films pas encore sortis en salle. Nous avons considéré que les films de 2023 avec des votes étaient déjà sortis. 
*   Séparer ces 2 DF en 2 autres, un avec un tconst unique (avec les infos sur les films, afin d'avancer sur l'analyse avec des données plus faciles à traiter), et un avec plusieurs tconst mais pour chaque film, des informations sur les acteurs, actrices et réalisateurs. 



1 - Limite des nombres de votes

In [ ]:
# Médiane de numVotes

df_merged = pd.read_csv('def_3.csv')
df_merged['numVotes'].median()

# Résultat : 673
# Choix de garder uniquement les films au dessus de 673 votes

2 - Suppression des duplicates

In [ ]:
# Suppression des lignes en doublon en se référant aux valeurs d'une colonne

df_merges.drop_duplicates(subset = ['tconst'], ignore_index = True, inplace=True)

3 - Création de 2 DataFrame : en salle / pas encore sortis

In [ ]:
# A partir du DF final comprenant les films sortis et pas encore sortis :

# Choix de drop les films au dessus de 2024, puis pour les films de 2023, choix de drop ceux qui ont des notes (forcément pas sortis)
df_ensalle = df_merged.drop(df_merged[df_merged['startYear'] >= 2023].index, inplace = False)
df_ensalle.dropna(subset=['averageRating', 'nconst'], inplace=True)

3 - Création des DataFrame film/crew

In [ ]:
# Suppression des colonnes inutiles pour avoir un DF film et un DF crew (sur le DF nouveautés et DF en salle)

col_a_supp = ['birthYear', 'language', 'knownForTitles','genres', 'nconst', 'primaryName', 'category']
df_films_ensalle = df_ensalle.drop(columns = col_a_supp, axis=1)

# 5 - Préparation pour les KPIs et le Machine Learning

1 - Get Dummies sur les genres

In [ ]:
# Get dummies puis concat 
genres = df_ensalle['genres'].str.get_dummies(',')
df_ensalle_genres = pd.concat([df_ensalle, genres], axis=1)

2 - Gestion des types et des colonnes

In [ ]:
# StartYear en INT (ici sur df_nouveaute, mais aussi fait sur le DF en salle)
df_nouveaute['startYear'] = df_nouveaute['startYear'].astype(int)

# deathYear en numérique, fillna "0", retypage en INT
df_nouveaute['deathYear'] = pd.to_numeric(df_nouveaute['deathYear'], errors='coerce')
df_nouveaute['deathYear'].fillna('0', inplace = True)
df_nouveaute['deathYear'] = df_nouveaute['deathYear'].astype(int)

In [ ]:
# Renommer colonne genre N/
df_ensalle_genres.rename(columns={'\\N':'UnknownGenre'}, inplace=True)
# Renomer colonne df_ensalle Unknown Genre
df_ensalle_genres.rename(columns={'Unknown genre':'UnknownGenre'}, inplace=True)

3 - Boxplots 

In [ ]:
# Boxplots pour analyser les données visuellement

sns.boxplot(
    x = df["averageRating"]
)

sns.show()